In [3]:
import xml.etree.ElementTree as ET
import mysql.connector
import logging
import uuid


In [35]:
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="AmeSame420",
        database="mydb"
    )


ns = {'ns': 'http://www.battlescribe.net/schema/catalogueSchema'}


def insert_faction(root, cursor):
    _name = root.get('name')
    _description = root.get('description')
    if _description is None:
        _description = ""
    _id = root.get('id')

    sql = "INSERT INTO Faction (name, description, id) VALUES (%s, %s,%s)"
    try:
        cursor.execute(sql, (_name, _description, _id))
    except Exception as e:
        logging.error("Error at %s", "insert_faction", exc_info=e)
        print(e)

    return _id


def insert_unit2(cursor, root, faction_id):
    sharedSelectionEntries = root.find('ns:sharedSelectionEntries', ns)
    for selectionEntry in sharedSelectionEntries:
        if "[Legends]" not in selectionEntry.get("name"):
            if selectionEntry.get("type") == "model" or selectionEntry.get("type") == "unit":

                _name = selectionEntry.get("name")
                _description = selectionEntry.get("description")
                _description = ""
                _id = selectionEntry.get("id")

                sql = "INSERT INTO Unit (name, description, id, faction_id) VALUES (%s, %s,%s,%s)"
                try:
                    cursor.execute(sql, (_name, _description, _id, faction_id))
                    parse_selectionEntry(cursor, selectionEntry, _id)
                    insert_keywords(cursor, selectionEntry, _id)
                except Exception as e:
                    logging.error("Error at %s", "insert_unit", exc_info=e)
                    print(e)

def insert_keywords(cursor, selectionEntry, unit_id):
    keywords = selectionEntry.findall(".//ns:categoryLink", ns)
    for keyword in keywords:
        _name = keyword.get("name")
        _id = keyword.get("targetId")
        sql = "INSERT IGNORE INTO ModelKeyWord (text, id) VALUES (%s, %s)"
        sql_map = "INSERT IGNORE INTO unit_has_ModelKeyword (Unit_id, ModelKeyword_id) VALUES (%s, %s)"
        try:
            cursor.execute(sql, (_name, _id))
            cursor.execute(sql_map, (unit_id, _id))
        except Exception as e:
            logging.error("Error at %s", "insert_keywords", exc_info=e)
            print(e)

def parse_selectionEntry(cursor, selectionEntry, unit_id):
    profiles = selectionEntry.findall(".//ns:profile", ns)

    for profile in profiles:
        if profile.get("typeName") == "Unit" or profile.get("typeName") == "Model":
            insert_model(cursor, profile, unit_id)
        if profile.get("typeName") == "Abilities":
            insert_abilities(cursor, profile, unit_id)
        if profile.get("typeName") == "Ranged Weapons" or profile.get("typeName") == "Melee Weapons":
            insert_weapons(cursor, profile, unit_id)
            print("insert_weapons")

def insert_weapons(cursor, profile, unit_id):
    _name = profile.get("name")
    _id = profile.get("id")
    c = profile.find(".//ns:characteristics", ns)



    _range = None
    _strength = None
    _ap = None
    _damage = None
    _attack = None
    _hit = None


    for characteristic in c:
        if characteristic.get("name") == "Range":
            _range = characteristic.text
        if characteristic.get("name") == "S":
            _strength = characteristic.text
        if characteristic.get("name") == "AP":
            _ap = characteristic.text
        if characteristic.get("name") == "D":
            _damage = characteristic.text
        if characteristic.get("name") == "A":
            _attack = characteristic.text
        if characteristic.get("name") == "WS":
            _hit = characteristic.text
        if characteristic.get("name") == "BS":
            _hit = characteristic.text

    sql = "INSERT INTO weapon (name, range, hit_skill, strength, armor_piercing, damage, id, attack, unit_id) VALUES (%s, %s,%s,%s,%s,%s,%s,%s,%s)"
    print(_name, _range, _hit, _strength, _ap, _damage, _id, _attack, unit_id)
    try:
        cursor.execute(sql, (_name, _range, _hit, _strength, _ap, _damage, _id, _attack, unit_id))
    except Exception as e:
        logging.error("Error at %s", "insert_weapons", exc_info=e)
        print(e)

def insert_abilities(cursor, profile, unit_id):
    _name = profile.get("name")
    _id = profile.get("id")
    _description = profile.findall(".//ns:characteristic", ns)[0].text

    sql = "INSERT INTO modelability (name, description, id) VALUES (%s, %s,%s)"
    sql_map = "INSERT INTO unit_has_modelability (unit_id, ModelAbility_id) VALUES (%s, %s)"
    try:
        cursor.execute(sql, (_name, _description, _id))
        cursor.execute(sql_map, (unit_id, _id))
    except Exception as e:
        logging.error("Error at %s", "insert_abilities", exc_info=e)
        print(e)

def create_keywords_sql(cursor, keywords, _id):
    _id = str(uuid.uuid4())
    for keyword in keywords:
        # Check if the keyword already exists
        cursor.execute("SELECT id FROM weaponkeyword WHERE text = %s", (keyword,))
        result = cursor.fetchone()

        # If the keyword does not exist, insert it
        if result is None:
            cursor.execute("INSERT INTO weaponkeyword (id, text) VALUES (%s, %s)", (_id, keyword))


def insert_model(cursor, model, unit_id):
    _name = model.get("name")
    _id = model.get("id")

    move = None
    leadership = None
    save = None
    toughness = None
    wounds = None
    oc = None
    keywords = []
    for characteristics in model.findall(".//ns:characteristic", ns):

        if characteristics.get("name") == "M":
            move = characteristics.text
        elif characteristics.get("name") == "LD":
            leadership = characteristics.text
        elif characteristics.get("name") == "SV":
            save = characteristics.text
        elif characteristics.get("name") == "T":
            toughness = characteristics.text
        elif characteristics.get("name") == "W":
            wounds = characteristics.text
        elif characteristics.get("name") == "OC":
            oc = characteristics.text


        sql_models = "INSERT INTO Model (id, name, move, toughness,wounds, leadership, save, unit_id, oc) VALUES (%s, %s,%s,%s, %s, %s,%s,%s,%s)"


    try:
        cursor.execute(sql_models, (_id, _name, move, toughness, wounds, leadership, save, unit_id, oc))

    except Exception as e:
        logging.error("Error at %s", "insert_model", exc_info=e)
        print(e)


def parse_cat_file(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    return root


def ingest_data(root, cursor):
    faction_id = insert_faction(root, cursor)
    insert_unit2(cursor, root, faction_id)
    db_connection.commit()


def clear_all_tables(cursor):
    # Get all table names
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()

    # Iterate over all tables and delete all records
    for table in tables:
        try:
            cursor.execute(f"DELETE FROM `{table[0]}`")
        except mysql.connector.Error as err:
            print(f"Something went wrong: {err}")



In [36]:
db_connection = connect_to_db()
cursor = db_connection.cursor()


In [39]:
root = parse_cat_file("./wh40k-10e/Chaos - Death Guard.cat")
clear_all_tables(cursor)
ingest_data(root, cursor)

ERROR:root:Error at insert_weapons
Traceback (most recent call last):
  File "C:\Users\krist\AppData\Local\Programs\Python\Python310\lib\site-packages\mysql\connector\connection_cext.py", line 697, in cmd_query
    self._cmysql.query(
_mysql_connector.MySQLInterfaceError: You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'range, hit_skill, strength, armor_piercing, damage, id, attack, unit_id) VALUES ' at line 1

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\krist\AppData\Local\Temp\ipykernel_22716\1056863943.py", line 110, in insert_weapons
    cursor.execute(sql, (_name, _range, _hit, _strength, _ap, _damage, _id, _attack, unit_id))
  File "C:\Users\krist\AppData\Local\Programs\Python\Python310\lib\site-packages\mysql\connector\cursor_cext.py", line 372, in execute
    result = self._cnx.cmd_query(
  File "C:\Users\krist\AppDa

➤ Master-crafted manreaper - strike Melee 2+ 9 -2 3 8576-b61a-fda9-9ec6 5 e6a4-7055-fd23-5123
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'range, hit_skill, strength, armor_piercing, damage, id, attack, unit_id) VALUES ' at line 1
insert_weapons
➤ Master-crafted manreaper - sweep Melee 2+ 6 -1 1 a230-d987-b4fa-381b 10 e6a4-7055-fd23-5123
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'range, hit_skill, strength, armor_piercing, damage, id, attack, unit_id) VALUES ' at line 1
insert_weapons
Plague bolt pistol 12" 2+ 4 0 1 8877-9ef7-eb6b-2ac0 1 cfc1-1741-530c-3335
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'range, hit_skill, strength, armor_piercing, damage, id, attack, unit_id) VALU

ERROR:root:Error at insert_weapons
Traceback (most recent call last):
  File "C:\Users\krist\AppData\Local\Programs\Python\Python310\lib\site-packages\mysql\connector\connection_cext.py", line 697, in cmd_query
    self._cmysql.query(
_mysql_connector.MySQLInterfaceError: You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'range, hit_skill, strength, armor_piercing, damage, id, attack, unit_id) VALUES ' at line 1

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\krist\AppData\Local\Temp\ipykernel_22716\1056863943.py", line 110, in insert_weapons
    cursor.execute(sql, (_name, _range, _hit, _strength, _ap, _damage, _id, _attack, unit_id))
  File "C:\Users\krist\AppData\Local\Programs\Python\Python310\lib\site-packages\mysql\connector\cursor_cext.py", line 372, in execute
    result = self._cnx.cmd_query(
  File "C:\Users\krist\AppDa

1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'range, hit_skill, strength, armor_piercing, damage, id, attack, unit_id) VALUES ' at line 1
insert_weapons
Rothail volley gun 24" 3+ 5 0 1 dc7d-2eeb-97ac-2298 3 e58b-507a-937f-7abf
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'range, hit_skill, strength, armor_piercing, damage, id, attack, unit_id) VALUES ' at line 1
insert_weapons
Heavy slugger 36" 3+ 5 -1 1 4bdd-bf4b-b546-1b6a 4 e58b-507a-937f-7abf
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'range, hit_skill, strength, armor_piercing, damage, id, attack, unit_id) VALUES ' at line 1
insert_weapons
Entropy cannon 24" 3+ 10 -3 D6+1 45b1-85d6-9d92-a1ec 1 e58b-507a-937f-7abf
1064 (42000):